# 01 — Generate + Score (pipeline)

Notebook de orquestração: chama os scripts oficiais do repo para **gerar** `submission.csv` e **scorar** (rápido e strict).

Recomendação: antes de confiar em um resultado, use **Restart & Run All**.

Outputs ficam em `runs/notebook_runs/` (ignorado pelo git).


In [ ]:
from __future__ import annotations

import csv
import json
import shlex
import subprocess
import sys
import time
from pathlib import Path


def find_repo_root(start: Path) -> Path:
    start = start.resolve()
    for cand in (start, *start.parents):
        if (cand / "pyproject.toml").is_file():
            return cand
    return start


ROOT = find_repo_root(Path.cwd())
OUT_DIR = ROOT / "runs" / "notebook_runs"
OUT_DIR.mkdir(parents=True, exist_ok=True)


def run(cmd: list[str], *, capture: bool = False) -> subprocess.CompletedProcess[str]:
    print("+", " ".join(shlex.quote(c) for c in cmd))
    return subprocess.run(
        cmd,
        cwd=str(ROOT),
        text=True,
        capture_output=capture,
        check=True,
    )


def git_sha() -> str:
    try:
        proc = subprocess.run(
            ["git", "rev-parse", "HEAD"],
            cwd=str(ROOT),
            text=True,
            capture_output=True,
            check=True,
        )
        return (proc.stdout or "").strip()
    except Exception:
        return "unknown"


In [ ]:
# MODE:
# - "make_submit": usa `python -m santa_packing.cli.make_submit` (gera + score strict + arquiva)
# - "manual": chama `python -m santa_packing.cli.generate_submission` + `score_submission`
MODE = "make_submit"

# Comuns
PYTHON = sys.executable
NMAX = 200
NAME = "notebook"

# make_submit
CONFIG = "configs/submit.json"  # opcional; use "" para não passar --config
GEN_EXTRA_ARGS = ""  # ex.: "--sa-nmax 0" (CLI sobrescreve o config)

# manual
GEN_ARGS = "--sa-nmax 30 --sa-steps 400 --sa-batch 64"  # args extras para o gerador


In [ ]:
ts = time.strftime("%Y%m%d-%H%M%S")
sha = git_sha()

run_dir: Path | None = None
submission_path: Path
strict_score: dict

if MODE == "make_submit":
    cmd = [
        PYTHON,
        "-m",
        "santa_packing.cli.make_submit",
        "--nmax",
        str(int(NMAX)),
        "--name",
        NAME,
        "--submissions-dir",
        str(OUT_DIR),
    ]
    if CONFIG.strip():
        cmd += ["--config", CONFIG]
    cmd += ["--"] + shlex.split(GEN_EXTRA_ARGS)

    proc = run(cmd, capture=True)
    print(proc.stdout)

    for line in (proc.stdout or "").splitlines():
        if line.startswith("Run:"):
            run_dir = Path(line.split(":", 1)[1].strip())
            break
    if run_dir is None:
        raise RuntimeError("Não consegui extrair o run_dir do stdout do make_submit")

    submission_path = run_dir / "submission.csv"
    strict_score = json.loads((run_dir / "score.json").read_text(encoding="utf-8"))
else:
    submission_path = OUT_DIR / f"submission_{ts}.csv"
    cmd_gen = [
        PYTHON,
        "-m",
        "santa_packing.cli.generate_submission",
        "--out",
        str(submission_path),
        "--nmax",
        str(int(NMAX)),
    ] + shlex.split(GEN_ARGS)
    run(cmd_gen)

    cmd_strict = [
        PYTHON,
        "-m",
        "santa_packing.cli.score_submission",
        str(submission_path),
        "--nmax",
        str(int(NMAX)),
        "--pretty",
    ]
    proc_strict = run(cmd_strict, capture=True)
    strict_score = json.loads(proc_strict.stdout or "{}")

print("Strict score:", strict_score.get("score"))


In [ ]:
cmd_fast = [
    PYTHON,
    "-m",
    "santa_packing.cli.score_submission",
    str(submission_path),
    "--nmax",
    str(int(NMAX)),
    "--no-overlap",
]
proc_fast = run(cmd_fast, capture=True)
fast_score = json.loads(proc_fast.stdout or "{}")
print("Fast score:", fast_score.get("score"))

if run_dir is not None:
    (run_dir / "score_fast.json").write_text(json.dumps(fast_score, indent=2) + "
", encoding="utf-8")


In [ ]:
experiments_csv = OUT_DIR / "experiments.csv"

row = {
    "timestamp": ts,
    "git_sha": sha,
    "mode": MODE,
    "name": NAME,
    "nmax": int(NMAX),
    "config": CONFIG if MODE == "make_submit" else "",
    "gen_args": GEN_ARGS if MODE != "make_submit" else GEN_EXTRA_ARGS,
    "run_dir": str(run_dir) if run_dir is not None else "",
    "submission": str(submission_path),
    "score_strict": strict_score.get("score"),
    "s_max_strict": strict_score.get("s_max"),
    "score_fast": fast_score.get("score"),
    "s_max_fast": fast_score.get("s_max"),
}

fields = list(row.keys())
is_new = not experiments_csv.exists()

with experiments_csv.open("a", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=fields)
    if is_new:
        w.writeheader()
    w.writerow(row)

print("Logged:", experiments_csv)

# Show last 10 rows (no pandas)
rows = []
with experiments_csv.open("r", newline="", encoding="utf-8") as f:
    r = csv.DictReader(f)
    rows = list(r)

for r in rows[-10:]:
    print(r["timestamp"], r["mode"], r["score_strict"], r["submission"])
